Copied from notebook, need to tweak data/variables for out data.

In [ ]:
from sofima.processor import warp
import numpy as np
import tensorstore as ts

from connectomics.common import bounding_box
from connectomics.common import box_generator
from connectomics.volume import subvolume

In [ ]:
# Prep renderer to read data

GCS_BUCKET = 'TODO:YOUR_BUCKET'

class SyncAdapter:
  """Makes it possible to use a TensorStore objects as a numpy array."""
  def __init__(self, tstore):
    self.tstore = tstore

  def __getitem__(self, ind):
    return np.array(self.tstore[ind])

  def __getattr__(self, attr):
    return getattr(self.tstore, attr)

  @property
  def shape(self):
    return self.tstore.shape

  @property
  def ndim(self):
    return self.tstore.ndim


class StitchAndRender3dTiles(warp.StitchAndRender3dTiles):

  cache = {}

  def _open_tile_volume(self, tile_id: int):
    if tile_id in self.cache:
      return self.cache[tile_id]

    vol = ts.open({
      'driver': 'n5',
      'kvstore': {
          'driver': 'gcs',
          'bucket': GCS_BUCKET,
      },
      'path': self._tile_pattern_path.format(tile_id=tile_id)
    }).result().T  # convert to zyx axis layout
    self.cache[tile_id] = SyncAdapter(vol)
    return self.cache[tile_id]

In [ ]:
# Prep renderer inputs

# +---> x
# |
# v y
tile_layout = np.array([
    [4, 5, 14, 15],
    [3, 6, 13, 16],
    [2, 7, 12, 17],
    [1, 8, 11, 18],
    [0, 9, 10, 19],
])


In [ ]:
# Prep output buffer

ds_out = ts.open({
  'driver': 'n5',
  'kvstore': {
      'driver': 'gcs',
      'bucket': GCS_BUCKET,
  },
  'create': True,
  'delete_existing': True,
  'path': f'TODO:DATASET_PATH/stitched_st20',
  'metadata': {
    'compression': {'type': 'blosc', 'cname': 'lz4', 'clevel': 5, 'shuffle': 1},
    'dataType': 'uint8',
    'dimensions': [1590, 9740, 7650],
    'blockSize': [128, 128, 128],
    'axes': ['z', 'y', 'x'],
    'resolution': [13,10,10],
    'units': ['nm', 'nm', 'nm']
  },
}).result()

In [ ]:
# Initialize the renderer

renderer = StitchAndRender3dTiles(
    tile_map=tile_layout,
    tile_pattern_path='TODO:DATASET_PATH/{tile_id}_clahe',
    tile_mesh_path='solved_meshes_init_cl_20_80.npz',
    stride=(20, 20, 20),
    offset=[-250, -380, 90],
    parallelism=8
)

In [ ]:
# Call renderer.process on subvolumes solved meshes

box = bounding_box.BoundingBox(start=(0,0,0), size=ds_out.shape[::-1])
gen = box_generator.BoxGenerator(box, (512, 512, 512), (0, 0, 0), True)
renderer.set_effective_subvol_and_overlap((512, 512, 512), (0, 0, 0))

import time
for i, sub_box in enumerate(gen.boxes):
    t_start = time.time()
    inp_subvol = subvolume.Subvolume(np.zeros(sub_box.size[::-1], dtype=np.uint8)[None, ...], sub_box)
    ret_subvol = renderer.process(inp_subvol)
    t_render = time.time()
    ds_out[ret_subvol.bbox.to_slice3d()].write(ret_subvol.data[0, ...]).result()
    t_write = time.time()
    print('box {i}: {t1:0.2f} render  {t2:0.2f} write'.format(i=i, t1=t_render - t_start, t2=t_write - t_render))